---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)





In [2]:
def date_sorter():
    
    import numpy as np

    mes = dict(jan='01', feb='02', mar='03', apr='04', may='05', jun='06', jul='07', aug='08', sep='09', oct='10', nov='11', dec='12')
    
    def conversion(string):
        s = string.lower()[:3]
        return mes[s]
  

    def cambio(x):
        year=int(x)
        if (year>=0 and year<=9)  :   year=str(year); year=year.zfill(2); year='20'+ year;
        elif (year>=10 and year<=50): year=str(year); year='20'+ year;
        elif (year>=51 and year<=99): year=str(year); year='19'+ year;
        return (year)

    #Cases
    
    #Case 1: 03/25/93; 9/27/75; 2/6/96; 07/25/1984.
   
    format11=df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    format12=df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    format1= pd.concat([format11,format12])
    format1.reset_index(inplace=True)
    format1_index = format1['level_0']
    format1.columns=['level_0','match','month','day','year']
    format1['year']=format1['year'].apply(str)
    format1['year']=format1['year'].apply(lambda x: cambio(x) if len(x)<=2 else x)


    # Case 2: 22 Jan 1996;  14 Oct 1992; 30 May 2001;  May 14, 1989.
    
    format2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    format2.reset_index(inplace=True)
    format2_index = format2['level_0']

    format2[1] = format2[1].apply(lambda x: x.replace(',',''))
    format2['day'] = format2[1].apply(lambda x:x.split(' ')[0])
    format2['year'] = format2[1].apply(lambda x:x.split(' ')[1])
    format2[0] = format2[0].apply(lambda x: conversion(x))
    format2.columns=['level_0','match','month','day-year','day','year']
    format2.drop('day-year',axis=1,inplace=True) 



    # Case 3: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009; sep 2009;
    
    format3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    format3.reset_index(inplace=True)
    format3_index = format3['level_0']

    format3.columns=['level_0','match','day','month','year']
    format3['day'] = format3['day'].replace(np.nan,'01')
    format3['month'] = format3['month'].apply(lambda x: conversion(x))


    # Case 4:  6/2009 , 06/2009
    format4 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    format4.reset_index(inplace=True)
    format4_index= format4['level_0']

    l=[]
    for i in format4_index:
       if not(i in format1_index.values):
           l.append(i)
    l = np.asarray(l)

    format4 = format4[format4['level_0'].isin(l)]

    format4.columns=['level_0','match','month','year']
    format4['day']=1

            
    # Case 5:  2009; 2010
    format5= df.str.extractall(r'([1-2]\d{3})')


    format5.reset_index(inplace=True)
    format5_index = format5['level_0']


    m=[]
    for i in format5_index:
        if not((i in format2_index.values) | (i in format3_index.values) |  (i in format4_index.values)   ):
            m.append(i)
    m = np.asarray(m)
    format5 = format5[format5['level_0'].isin(m)]

    format5.columns=['level_0','match','year']
    format5['day']=1
    format5['month']=1
   
  
            
    bbdd = pd.concat([format1,format2,format3,format4,format5])
    
    
    bbdd['date'] =pd.to_datetime(bbdd['year'].apply(str)+'/'+bbdd['month'].apply(str)+'/'+bbdd['day'].apply(str))
         
    bbdd = bbdd.sort_values(by='level_0').set_index('level_0')
   
    lista = bbdd['date']
   
    answer = pd.Series([i[0] for i in sorted(enumerate(lista), key=lambda x:x[1])])
    
    return (answer)

date_sorter()


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64